# iscas_2025

## Setup

In [ ]:
#!pip install qiskit qiskit-aer qiskit-ibm-runtime

In [ ]:
import numpy as np
import random
import string
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer, AerSimulator
from qiskit_ibm_runtime.fake_provider import FakeVigoV2

## Permutation

In [ ]:
def shift_right(s):
  """
  Shifts the bits of a bitstring one place to the right.
  """
  return s[-1] + s[:-1]

## Grover

In [ ]:
def grover(n_qubits, oracle, iterations=1, theta=np.pi/2):
  """
  Implements an approximate Grover search algorithm with a parameterized circuit.

  Args:
      n_qubits: The number of qubits in the circuit.
      oracle: The oracle function that marks the solution state.
      iterations: The number of iterations of the Grover operator.
      theta: The angle parameter for the Grover operator.

  Returns:
      A QuantumCircuit implementing the approximate Grover search.
  """
  qr = QuantumRegister(n_qubits)
  cr = ClassicalRegister(n_qubits)
  qc = QuantumCircuit(qr, cr)
  # Initialization
  for i in range(n_qubits):
    qc.h(qr[i])
  # Grover operator
  for _ in range(iterations):
    qc.append(oracle, qr)
    # Diffusion operator
    for i in range(n_qubits):
      qc.h(qr[i])
    for i in range(n_qubits):
      qc.x(qr[i])
    qc.h(qr[n_qubits - 1])
    qc.mcx(list(range(n_qubits - 1)), qr[n_qubits - 1])
    qc.h(qr[n_qubits - 1])
    for i in range(n_qubits):
      qc.x(qr[i])
    for i in range(n_qubits):
      qc.h(qr[i])
  qc.measure(qr, cr)

  return qc

In [ ]:
def create_oracle(s):
  """
  Creates a Grover's oracle for a specific bitstring.

  Args:
      s: The bitstring to be marked by the oracle.

  Returns:
      A QuantumCircuit representing the oracle.
  """
  n_qubits = len(s)
  qr = QuantumRegister(n_qubits)
  qc = QuantumCircuit(qr)
  # Apply X gates to flip qubits corresponding to '0' in the target bitstring
  for i in range(n_qubits):
    if s[i] == '0':
      qc.x(qr[i])
  # Apply multi-controlled-Z gate to flip the phase of the target state
  qc.h(n_qubits-1)
  qc.mcx(list(range(n_qubits-1)), n_qubits-1)
  qc.h(n_qubits-1)
  # Apply X gates again to revert the initial flips
  for i in range(n_qubits):
    if s[i] == '0':
      qc.x(qr[i])

  return qc

In [ ]:
def simulate(circ,backend):
  """
  Simulate a quantum circuit on a given backend.

  Args:
      circ: qiskit QuantumCircuit to be executed.
      backend: qiskit Aer backend to run the circuit on

  Returns:
      counts of the measurement results
  """
  qc = transpile(circ, backend)
  job = backend.run(qc, shots=1024)
  return job.result().get_counts(circ)

In [ ]:
def permutation_grover(t, permutation_function):
  """
  Finds the original bitstring 's' given a permuted bitstring 't' and the permutation function.

  Args:
      t: The permuted bitstring.
      permutation_function: A function that applies the permutation to a bitstring.

  Returns:
      The original bitstring 's'.
  """
  n_qubits = len(t)
  # Define the oracle that checks if a given candidate bitstring 's' produces 't' when permuted.
  def oracle_function(candidate_s):
    permuted_candidate = permutation_function(candidate_s)
    return permuted_candidate == t

  # Iterate through all possible bitstrings and check if they produce 't' when permuted.
  def create_oracle_from_function(oracle_function):
    for i in range(2**n_qubits):
      candidate_s = bin(i)[2:].zfill(n_qubits)
      if oracle_function(candidate_s):
        oracle_circ = create_oracle(candidate_s)
        return oracle_circ
    return None

  oracle_circ = create_oracle_from_function(oracle_function)

  if oracle_circ is not None:
    grover_circuit = grover(n_qubits, oracle_circ, iterations=1)
    counts = simulate(grover_circuit, backend)
    most_frequent_outcome = max(counts, key=counts.get)

    best_iterations = 0
    best_probability = 0
    for iterations in range(1, int(np.sqrt(2**len(t)))):
      grover_circuit = grover(n_qubits, oracle_circ, iterations=iterations)
      counts = simulate(grover_circuit, backend)
      if counts:
        most_frequent_outcome = max(counts, key=counts.get)
        probability = counts[most_frequent_outcome] / 1024
        if probability > best_probability:
          best_probability = probability
          best_iterations = iterations
    return most_frequent_outcome, best_iterations

In [ ]:
backend = AerSimulator()
s = "10000"
t = shift_right(s)
s = permutation_grover(t, shift_right)[0]
best_iterations = permutation_grover(t, shift_right)[1]

print(f"The permuted bitstring t is: {t}")
if s:
  print(f"The original bitstring s is: {s[::-1]}")
else:
  print("No solution found.")
if best_iterations:
  print(f"The best number of Grover iterations is: {best_iterations}")
else:
  print("No solution found.")


In [ ]:
backend = FakeVigoV2()
s = "10000"
t = shift_right(s)
s = permutation_grover(t, shift_right)[0]
best_iterations = permutation_grover(t, shift_right)[1]

print(f"The permuted bitstring t is: {t}")
if s:
  print(f"The original bitstring s is: {s[::-1]}")
else:
  print("No solution found.")
if best_iterations:
  print(f"The best number of Grover iterations is: {best_iterations}")
else:
  print("No solution found.")

## Quantum walk

In [ ]:
def walk(n,steps):
  num_qubits = int(np.ceil(np.log2(n)))
  qc = QuantumCircuit(num_qubits)
  # Initialize the walk at 1
  #qc.x(0)
  # Bias angle, greater than pi/2 favors right movement more
  theta = np.pi / 4
  for _ in range(steps):
    # Apply biased rotation (coin flip)
    for qubit in range(num_qubits):
        qc.ry(theta, qubit)

    # Apply a controlled NOT to shift the walker to the next position
    for qubit in range(num_qubits - 1):
        qc.cx(qubit, qubit + 1)
  qc.measure_all()
  return qc

In [ ]:
n = 5
steps = n-1
backend = AerSimulator()
T = []
for step in range(1,steps+1):
  qc = walk(n,step)
  counts = simulate(qc,backend)
  if sorted(counts.items(), key=lambda item: item[1], reverse=True)[0][0] == bin(5)[2:].zfill(3):
    T.append(step)
print(min(T))

## Total variation distance

In [ ]:
s = "10000"
t = shift_right(s)
qc = grover(5,create_oracle(t))
counts =  simulate(qc,AerSimulator())
noisy_counts = simulate(qc,FakeVigoV2())

In [ ]:
difference_counts = {}
for key in counts:
  if key in noisy_counts:
    difference_counts[key] = counts[key] - noisy_counts[key]
  else:
    difference_counts[key] = counts[key]

In [ ]:
total_variation_distance = sum(abs(value) for value in difference_counts.values())
print(total_variation_distance/1024)

## Cesar's cipher

In [ ]:
def generate_random_string(length):
  """
  Generates a random string of lowercase letters.

  Args:
      length (int): The desired length of the random string.

  Returns:
      str: A random string of lowercase letters with the specified length.
  """
  letters = string.ascii_lowercase
  return ''.join(random.choice(letters) for i in range(length))

In [ ]:
def cesars_cipher(text, shift_amount):
  """
  Encrypts or decrypts a text using a Caesar's cipher based on the shift_right function.

  Args:
      text: The text to be encrypted or decrypted.
      shift_amount: The number of positions to shift each character.

  Returns:
      The encrypted or decrypted text.
  """
  result = ""
  for char in text:
    if char.isalpha():
      start = ord('a') if char.islower() else ord('A')
      shifted_char_ord = (ord(char) - start + shift_amount) % 26 + start
      result += chr(shifted_char_ord)
    else:
      result += char
  return result

### Classical frequency attack

In [ ]:
def frequency_analysis(ciphertext, top_letter="e"):
  """
  Performs frequency analysis on a given ciphertext and attempts to decrypt it using letter frequency.

  Args:
      ciphertext (str): The encrypted text to analyze and decrypt.
      top_letter (str, optional): The letter assumed to be the most frequent in the plaintext.
                                  Defaults to 'e', which is the most common letter in English.

  Returns:
      str: The decrypted plaintext obtained by shifting the characters of the ciphertext.
  """
  letter_frequencies = {}
  for letter in ciphertext:
    if letter.isalpha():
      letter = letter.lower()
      if letter in letter_frequencies:
        letter_frequencies[letter] += 1
      else:
        letter_frequencies[letter] = 1

  most_frequent_letter = max(letter_frequencies, key=letter_frequencies.get)
  shift_amount = ord(most_frequent_letter) - ord(top_letter)

  plaintext = ""
  for letter in ciphertext:
    if letter.isalpha():
      start = ord('a') if letter.islower() else ord('A')
      shifted_letter_ord = (ord(letter) - start - shift_amount) % 26 + start
      plaintext += chr(shifted_letter_ord)
    else:
      plaintext += letter

  return plaintext

In [ ]:
key = 1
N = 100
text_list = [generate_random_string(5) for _ in range(N)]
when_correct = 0

In [ ]:
for text in text_list:
  encrypted_text = cesars_cipher(text, key)
  decrypted_text = frequency_analysis(encrypted_text)
  if decrypted_text == text:
    when_correct += 1
when_correct/N

### Quantum walk attack

In [ ]:
def generate_random_binary_string(length):
  """
  Generates a random binary string of a specified length.

  Args:
      length: The desired length of the binary string.

  Returns:
      A random binary string.
  """
  return ''.join(random.choice(['0', '1']) for _ in range(length))

In [ ]:
key = 1
N = 100
string_list = [generate_random_binary_string(5) for _ in range(N)]

In [ ]:
n = 5
steps = n-1
backend = AerSimulator()
when_correct = 0
T = []
for string in string_list:
  for step in range(1,steps+1):
    qc = walk(2**n,step)
    counts = simulate(qc,backend)
    string_perm = shift_right(string)
    string_rec = sorted(counts.items(), key=lambda item: item[1], reverse=True)[0][0]
    #print(string_perm, string_rec)
    if string_perm == string_rec:
      T.append(step)
  if T != []:
    t = min(T)
  else:
    t = 0
  #print(t)
  if string_perm == string_rec:
    when_correct += 1
when_correct/N